# Benchmark deepBlink

In [ ]:
import json
import numpy as np
import requests

from deepblink.io import load_model
from deepblink.inference import predict
from itertools import chain
from pathlib import Path

from piscis.data import load_datasets, transform_subdataset
from piscis.metrics import compute_metrics
from piscis.utils import pad_and_stack

In [ ]:
# Define path to outputs folder.
outputs_path = Path().absolute().parent / 'outputs'

# Define paths to datasets.
datasets_path = outputs_path / 'datasets'
piscis_datasets_path = datasets_path / 'piscis'
deepblink_datasets_path = datasets_path / 'deepblink'

# Define path to deepBlink models.
deepblink_models_path = outputs_path / 'deepblink_models'

# Define path to benchmarks.
benchmarks_path = outputs_path / 'benchmarks'
benchmarks_path.mkdir(parents=True, exist_ok=True)

### Run and benchmark deepBlink on Piscis dataset.

In [ ]:
# Create a dictionary for F1 scores.
f1s = {}

In [ ]:
# List subdatasets within the composite dataset.
dataset = '20230905'
subdataset_paths = Path(piscis_datasets_path / dataset).glob('*.npz')

In [ ]:
dataset_f1s = {}
f1s[dataset] = dataset_f1s
for subdataset_path in subdataset_paths:

    subdataset = subdataset_path.stem

    # Load the test dataset.
    test_ds = load_datasets(subdataset_path, load_train=False, load_valid=False, load_test=True)['test']
    images = pad_and_stack(test_ds['images']).squeeze()
    test_ds['images'] = images
    coords = test_ds['coords']

    subdataset_f1s = {}
    dataset_f1s[subdataset] = subdataset_f1s

    for deepblink_model_path in deepblink_models_path.glob(f'*{dataset}*.h5'):

        # Load the deepBlink model.
        model = load_model(deepblink_model_path)
    
        cell_size_f1s = []
        subdataset_f1s[deepblink_model_path.stem.split(f'{dataset}_')[-1]] = cell_size_f1s
    
        for image, c in zip(images, coords):

            # Run the deepBlink model.
            c_pred = predict(image, model)
        
            if (c_pred.size > 0) and (c.size > 0):
                f1 = compute_metrics(c_pred, c, distance_thresholds=np.linspace(0, 3, 50))['f1']
            else:
                f1 = 0
                
            cell_size_f1s.append(f1)

    with open(benchmarks_path / 'deepblink_f1s.json', 'w') as f:
        json.dump(f1s, f)

### Run and benchmark deepBlink on deepBlink datasets.

In [ ]:
for deepblink_dataset_path in deepblink_datasets_path.glob('*.npz'):

    deepblink_dataset = deepblink_dataset_path.stem

    # Load the test dataset.
    test_ds = load_datasets(deepblink_dataset_path, load_train=False, load_valid=False, load_test=True)['test']
    test_ds = transform_subdataset(test_ds, (512, 512), min_spots=1)
    images = test_ds['images'].squeeze()
    coords = test_ds['coords']

    # Load the deepBlink model.
    model = load_model(deepblink_models_path / f'deepblink_{deepblink_dataset}.h5')

    deepblink_dataset_f1s = []
    f1s[deepblink_dataset] = deepblink_dataset_f1s

    for image, c in zip(images, coords):

        # Run the deepBlink model.
        c_pred = predict(image, model)
    
        if (c_pred.size > 0) and (c.size > 0):
            f1 = compute_metrics(c_pred, c, distance_thresholds=np.linspace(0, 3, 50), return_df=False)['f1']
        else:
            f1 = 0
            
        deepblink_dataset_f1s.append(f1)

    with open(benchmarks_path / 'deepblink_f1s.json', 'w') as f:
        json.dump(f1s, f)